# Data Download from GDrive

In [ ]:
# Step 1: Install pydrive

# !pip install pydrive

In [ ]:
# url = 'https://drive.google.com/drive/u/1/folders/1SPuEOjOZIOm8mJkOqZKB7r0Bw1qgOMh5'

In [ ]:
# # Step 2: Use gdown to download the folder
# import gdown


# # Convert the folder URL to a format that gdown can use
# folder_id = url.split('/')[-1]
# gdown.download_folder(f'https://drive.google.com/drive/folders/{folder_id}', quiet=False)

# Harmonization Parameters Finder

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from datetime import datetime

## Read Data

In [ ]:
def read_sensor_data(csv_list):
    # csv_list = glob.glob('Lauren purpleair data\\*\\*.csv')

    csv_list
    sensor_dataframes = []

    for idx,csv_path in enumerate(csv_list):
            
        sensor_df = pd.read_csv(csv_path)
        
        if len(sensor_df.columns) < 4:
            sensor_df = pd.read_csv(csv_path,delimiter=';')
        sensor_dataframes.append(sensor_df)
    return sensor_dataframes

csv_list = glob.glob('Lauren purpleair data\\*\\*.csv')
sensor_dataframes = read_sensor_data(csv_list) #[:10]

## Clean Data

In [ ]:
def create_units_dict(df):
    units_dict = {}
    for column in df.columns:
        units_dict[column] = df.loc[0, column]
    return units_dict

units_dict = create_units_dict(sensor_dataframes[0])

In [ ]:
def clean_sensor_dataframes(sensor_dataframes):

    numeric_columns = ['pm2.5_atm_b', 'pm2.5_cf_1', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b',	'pm10.0_atm', 'pm10.0_atm_a', 'pm10.0_atm_b', 'pm10.0_cf_1', 'pm10.0_cf_1_a', 'pm10.0_cf_1_b']
    
    
    cleaned_dataframes = []
    for idx,sensor_df in enumerate(sensor_dataframes):
        # remove header
        sensor_df = sensor_df.iloc[1:]
        # Sort by 'Timestamp'
        try:
            sensor_df = sensor_df.sort_values(by=['time_stamp']).reset_index()
        except:
            print(idx)
            display(sensor_df)
            break
        # Change 'Timestamp' to date format
        sensor_df['time_stamp'] = pd.to_datetime(sensor_df['time_stamp'])



        # Filter rows based on timestamp
        # sensor_df = sensor_df[sensor_df['time_stamp'] <= '2024-02-10']

        # Convert the 'time_stamp' column to datetime
        sensor_df['time_stamp'] = pd.to_datetime(sensor_df['time_stamp']).dt.date
        # Perform the comparison using datetime
        comparison_date = datetime(2024, 2, 10).date()
        sensor_df = sensor_df[sensor_df['time_stamp'] <= comparison_date]
        
        try:
            sensor_df = sensor_df.sort_values(by=['time_stamp']).reset_index()
        except:
            print(idx)
            print(sensor_df)
            break

            # Convert specified columns to numeric
        sensor_df[numeric_columns] = sensor_df[numeric_columns].apply(pd.to_numeric, errors='coerce')
        # Append the cleaned DataFrame to the list
        cleaned_dataframes.append(sensor_df)
    return cleaned_dataframes

cleaned_sensor_dataframes = clean_sensor_dataframes(sensor_dataframes)

In [ ]:
cleaned_sensor_dataframes[12]

In [ ]:
sensor_names = [csv_path.split('\\')[1] for csv_path in csv_list] #[:10]

In [ ]:
def aggregate_sensor_data(sensor_dataframes, column_name):
    # Extracting the specified column from each sensor dataframe
    cutoff_value = len(sensor_dataframes[3])
    selected_columns = [df[column_name] for df in sensor_dataframes]
    # Combining these columns into a new dataframe
    combined_df = pd.concat(selected_columns, axis=1)[:cutoff_value]
    combined_df.columns = [f'Sensor{i+1}' for i in range(len(sensor_dataframes))]
    return combined_df

### Plot

In [ ]:
def plot_sensor_data(column_name, sensor_dataframes, sensor_names, units_dict, window_size=10):
    # Calculate the rolling median
    df = aggregate_sensor_data(sensor_dataframes,column_name)
    rolling_median = df.rolling(window=10, min_periods=1).median()
    for sensor_df, sensor_name in zip(sensor_dataframes, sensor_names):
        plt.plot(sensor_df['time_stamp'], sensor_df[column_name], label=sensor_name)
        
    plt.plot(sensor_dataframes[3]['time_stamp'], rolling_median.median(axis=1), color='black', linestyle='dashed', label='Median')
    plt.xlabel('time_stamp')
    plt.ylabel(f'{column_name} ({units_dict[column_name]})')
    plt.title(f'{column_name} Data for Sensors')
    plt.legend()
    plt.xticks(rotation=20, ha='right')
    plt.grid(True)
    plt.show()

plot_sensor_data('pm2.5_atm_b', cleaned_sensor_dataframes, sensor_names, units_dict, 10)

### Harmonization Parameters

In [ ]:
from sklearn.linear_model import LinearRegression

# Function to fit each sensor's data to the linear function
def fit_sensor_to_median(x, y):
    lr = LinearRegression()
    lr.fit(x, y)
    return lr.coef_[0], lr.intercept_  # Return the coefficients and intercept of the linear regression model

def fit_sensor_data(column_name, sensor_dataframes, sensor_names, units_dict, window_size=10, num_dataframes=1):
    plt.figure(figsize=(10, 6))  

    cutoff_value = 1000 #len(sensor_dataframes[3]) - 100

    num_dataframes = num_dataframes
    sensor_dataframes_working = sensor_dataframes[:num_dataframes]
    sensor_names_wokring = sensor_names[:num_dataframes]

    df = aggregate_sensor_data(sensor_dataframes, column_name)
    rolling_median = df.rolling(window=window_size, min_periods=1).median()
    fitted_data_dict = {}
    coefficients_data = []

    for sensor_df, sensor_name in zip(sensor_dataframes_working, sensor_names_wokring):
        X = sensor_df[column_name].to_frame(name=column_name)[:cutoff_value]
        Y = rolling_median.median(axis=1).to_frame('median')[:cutoff_value]
        coefficients, intercept = fit_sensor_to_median(X, Y)
        fitted_data = X * coefficients + intercept
        fitted_data = fitted_data.values.flatten()
        fitted_data_dict[sensor_name] = fitted_data
        
        # Append coefficients and intercept to list
        coefficients_data.append({
            'Sensor': sensor_name,
            'Coefficient': coefficients[0],
            'Intercept': intercept[0]
        })

    # Create DataFrame from the list of coefficients and intercepts
    coefficients_df = pd.DataFrame(coefficients_data)
    return coefficients_df

In [ ]:
numeric_columns = ['pm2.5_atm_b', 'pm2.5_cf_1', 'pm2.5_cf_1_a', 'pm2.5_cf_1_b',	'pm10.0_atm', 'pm10.0_atm_a', 'pm10.0_atm_b', 'pm10.0_cf_1', 'pm10.0_cf_1_a', 'pm10.0_cf_1_b']

for column_name in numeric_columns:
    coefficients_df = fit_sensor_data(column_name, cleaned_sensor_dataframes, sensor_names, units_dict, window_size=1, num_dataframes= 15) #len(sensor_dataframes))
    filename = f"{column_name.lower().replace(' ', '_')}_coefficients_df.csv"
    coefficients_df.to_csv(filename, index=False)
    print(f"{filename} saved successfully.")